# Data Preprocess
## Set up

In [1]:
import pandas as pd
pd.options.display.max_columns = None

## Getting data
Loading in data and add correct column names

In [2]:
df_fromgraduation = pd.read_csv("incubator_project_graduated_from_graduation.csv", header=None)
df_all = pd.read_csv("incubator_project_metrics_graduated_retired.csv", header=None)

In [3]:
CSV_HEADERS = ['project',
                'COM-1', 'COM-2',
                'POP-1',
                'STA-1','STA-2',
                'STA-3','STA-4','STA-5','STA-6',
                'STA-7','STA-8','STA-9',
                'TEC-1','TEC-2.1','TEC-2.2','TEC-2.3','TEC-2','TEC-3','TEC-4',
                'SWQ-1',
                'SWQ-2.1','SWQ-2.2','SWQ-2.3','SWQ-2.4','SWQ-2.5','SWQ-2.6','SWQ-2.7',
                'init','end','frequency (weeks)',
                'project_url']

df_fromgraduation.columns = CSV_HEADERS
df_all.columns = CSV_HEADERS

The columns: TEC-2.1, TECT-2.2 and TEC-2.3 are empty and/or removed from the paper.

In [4]:
df_fromgraduation = df_fromgraduation.drop(columns=["TEC-2.1",'TEC-2.2','TEC-2.3'])
df_all = df_all.drop(columns=["TEC-2.1",'TEC-2.2','TEC-2.3'])

Checking the 2 dataframes have the correct data and headers

In [5]:
df_fromgraduation.head(3)

,project,COM-1,COM-2,POP-1,STA-1,STA-2,STA-3,STA-4,STA-5,STA-6,STA-7,STA-8,STA-9,TEC-1,TEC-2,TEC-3,TEC-4,SWQ-1,SWQ-2.1,SWQ-2.2,SWQ-2.3,SWQ-2.4,SWQ-2.5,SWQ-2.6,SWQ-2.7,init,end,frequency (weeks),project_url
0,ace,9788017.927576544,1579,77,2010-05-26 23:38:08,9,23,2,2,True,-4,67,10,0,4022,19,1858,0.0010720897460694647,19.66,42,0,6,15,8.611501,235,2011-12-21,2020-06-17,12,https://github.com/apache/ace
1,apisix,901329.6034821316,27592,30442,2019-04-10 02:02:05,125,2452,-223,5,False,-15,1044,383,2413,67462,2162,2406,0.009605313577723847,7.48,41,1,0,12,26.869675,415,2020-07-15,2024-07-12,12,https://github.com/apache/apisix
2,abdera,2938138.296738281,335,61,2009-05-21 03:21:27,34,25,1,2,True,-2,121,11,0,26165,35,1611,0.017543859649122806,55.18,40,1,0,2,9.878605,274,2008-11-08,2020-11-26,12,https://github.com/apache/abdera


In [6]:
df_all.head(3)

,project,COM-1,COM-2,POP-1,STA-1,STA-2,STA-3,STA-4,STA-5,STA-6,STA-7,STA-8,STA-9,TEC-1,TEC-2,TEC-3,TEC-4,SWQ-1,SWQ-2.1,SWQ-2.2,SWQ-2.3,SWQ-2.4,SWQ-2.5,SWQ-2.6,SWQ-2.7,init,end,frequency (weeks),project_url
0,ace,9788017.927576544,644,77,2010-05-26 23:38:08,-15,23,0,2,True,1,40,4,0,4022,19,1858,0.0010720897460694647,19.66,42,0,6,15,8.611501,235,2009-04-24,2011-12-21,12,https://github.com/apache/ace
1,abdera,2938138.296738281,608,61,2009-05-21 03:21:27,286,25,0,2,True,0,76,5,0,26165,35,1611,0.017543859649122806,55.18,40,1,0,2,9.878605,274,2006-06-06,2008-11-08,12,https://github.com/apache/abdera
2,airavata,6952636.983029673,1571,324,2014-01-29 08:00:06,-266,114,0,5,False,3,61,3,1112,46472,303,10526,0.015394894854222536,2.27,0,50,95,340,49.897383,323,2011-05-05,2012-09-19,12,https://github.com/apache/airavata


## Adding status column and combing dataset

projects in "fromgraduation" = Graduated
project is in "all" but not in "graduated" = Retired

In [7]:
df_fromgraduation["status"] = "graduated"

In [8]:
df_all["status"] = df_all["project"].isin(df_fromgraduation["project"]).map({True: "graduated", False: "retired"})
df_retired = df_all[df_all["status"] == "retired"].copy()

Now we have the following dataframes:
- df_fromgraduation: graduated projects (sustainable)
- df_retired: retired projects (unsustainable)

We combine all the dataframes to one for further processing:

In [9]:
df_combined = pd.concat([df_retired, df_fromgraduation], ignore_index=True)

## Data Clean up

## Missing Data
Some of the projects are missing data, with some cells noted as "missing" in the dataframe. After discussion with supervisor it was decided to remove projects with missing data from the analysis.

In [10]:
count_missing = df_combined.apply(lambda row: row.astype(str).str.contains('missing', case=False, na=False).any(), axis=1).sum()

print(f"Number of rows containing 'missing': {count_missing}")

Number of rows containing 'missing': 42


In [11]:
df_combined = df_combined[~df_combined.apply(lambda row: row.astype(str).str.contains('missing', case=False, na=False).any(), axis=1)]


Checking for NaN values:

In [12]:
df_combined[df_combined.isna().any(axis=1)]

,project,COM-1,COM-2,POP-1,STA-1,STA-2,STA-3,STA-4,STA-5,STA-6,STA-7,STA-8,STA-9,TEC-1,TEC-2,TEC-3,TEC-4,SWQ-1,SWQ-2.1,SWQ-2.2,SWQ-2.3,SWQ-2.4,SWQ-2.5,SWQ-2.6,SWQ-2.7,init,end,frequency (weeks),project_url,status
98,cordova,1748916.7862903227,1151,1264,2018-02-13 10:02:11,0,62,0,1,True,0,73,11,2,36686,24,25,0.0,0.00,0,0,0,0,NaN,0,2012-10-17,2024-07-08,12,https://github.com/apache/cordova,graduated
122,Felix,5142729.431256367,25233,896,2009-05-20 02:00:07,310,322,0,2,False,-7,1365,60,0,69377,2303,35027,0.038252949699855106,0.00,0,0,0,0,NaN,0,2007-04-01,2024-07-09,12,https://github.com/apache/Felix,graduated
231,Sling,2693158.6690781005,42692,698,2009-03-27 15:41:54,1065,260,0,1,False,-8,810,53,0,81086,3266,46606,0.053478048667115995,0.00,0,0,0,0,NaN,0,2009-06-17,2024-08-14,12,https://github.com/apache/Sling,graduated
274,streams,NaN,18,197,2014-03-18 07:00:06,19,47,-18,1,False,-1,14,9,884,52879,230,1804,0.0,27.44,33,0,2,1,31.996976,112,2017-07-19,2024-03-21,12,https://github.com/apache/streams,graduated


In [13]:
df_combined = df_combined.dropna()

### Type processing

In [14]:
df_combined.dtypes

project               object
COM-1                 object
COM-2                 object
POP-1                 object
STA-1                 object
STA-2                 object
STA-3                 object
STA-4                 object
STA-5                  int64
STA-6                 object
STA-7                 object
STA-8                 object
STA-9                 object
TEC-1                 object
TEC-2                 object
TEC-3                 object
TEC-4                 object
SWQ-1                 object
SWQ-2.1              float64
SWQ-2.2                int64
SWQ-2.3                int64
SWQ-2.4                int64
SWQ-2.5                int64
SWQ-2.6              float64
SWQ-2.7                int64
init                  object
end                   object
frequency (weeks)      int64
project_url           object
status                object
dtype: object

**String columns**: Identifiers like `project` and `project_url` are treated as text.

**Numeric columns**: Fields representing counts or measurements — including stability metrics (`STA-2` to `STA-4`, `STA-7` to `STA-9`), technical activity metrics (`TEC-1` to `TEC-4`), and code quality metrics (`SWQ-2.1` to `SWQ-2.7`) — are converted to numeric types (`int64` or `float64`) using `pd.to_numeric`.

**Boolean column**: `STA-6` contains string values (`'True'`/`'False'`) which are normalized (lowercased and stripped) and mapped to boolean values. The column is converted to the nullable `boolean` dtype to support potential missing values.

**Datetime columns**: `init` and `end` are parsed as `datetime64[ns]` using `pd.to_datetime` for accurate time-based operations.

**Categorical column**: `status` contains a small set of repeating strin eitherels like `"reored"` and `"gra, which are well-suited for the `category` type (saves memory and improves performance in grouping/filtering).nalysis.


In [15]:
# Convert to String
string_cols = ['project', 'project_url']
df_combined[string_cols] = df_combined[string_cols].astype(str)

# Convert to Numeric
numeric_cols = [
    'STA-2', 'STA-3', 'STA-4', 'STA-7', 'STA-8', 'STA-9',
    'TEC-1', 'TEC-2', 'TEC-3', 'TEC-4',
    'SWQ-2.1', 'SWQ-2.2', 'SWQ-2.3', 'SWQ-2.4', 'SWQ-2.5', 'SWQ-2.6', 'SWQ-2.7',
    'COM-1', 'COM-2', 'POP-1', 'SWQ-1' 
]
df_combined[numeric_cols] = df_combined[numeric_cols].apply(pd.to_numeric)

# Convert to datetime
df_combined['init'] = pd.to_datetime(df_combined['init'])
df_combined['end'] = pd.to_datetime(df_combined['end'])

# Convert to boolean
df_combined['STA-6'] = df_combined['STA-6'].astype(str).str.strip().str.lower().map({
    'true': True,
    'false': False
})
df_combined['STA-6'] = df_combined['STA-6'].astype('boolean') 

# Convert status to category
df_combined['status'] = df_combined['status'].astype('category')

In [16]:
df_combined.dtypes

project                      object
COM-1                       float64
COM-2                         int64
POP-1                         int64
STA-1                        object
STA-2                         int64
STA-3                         int64
STA-4                         int64
STA-5                         int64
STA-6                       boolean
STA-7                         int64
STA-8                         int64
STA-9                         int64
TEC-1                         int64
TEC-2                         int64
TEC-3                         int64
TEC-4                         int64
SWQ-1                       float64
SWQ-2.1                     float64
SWQ-2.2                       int64
SWQ-2.3                       int64
SWQ-2.4                       int64
SWQ-2.5                       int64
SWQ-2.6                     float64
SWQ-2.7                       int64
init                 datetime64[ns]
end                  datetime64[ns]
frequency (weeks)           

### STA-1

STA-1 should contain information about how long the project has been running. But the column only has a single date. Since we have "init" and "end" we can change this date our with the number of days between init and end.

In [17]:
df_combined["STA-1"]

0      2011-12-04 19:45:29
1      2016-02-12 16:10:05
2      2017-04-13 19:06:57
3      2017-09-27 20:52:34
4      2016-08-28 07:00:07
              ...         
272    2014-06-07 07:00:10
273    2016-06-28 07:00:03
279    2017-09-01 07:00:11
280    2019-12-14 23:27:55
282    2017-12-20 08:00:09
Name: STA-1, Length: 237, dtype: object

In [18]:
df_combined["STA-1"] = (df_combined["end"] - df_combined["init"]).dt.days / 365.25

In [19]:
df_combined["STA-1"]

0      1.193703
1      2.182067
2      0.772074
3      2.028747
4      1.842574
         ...   
272    9.579740
273    5.828884
279    5.248460
280    1.667351
282    5.251198
Name: STA-1, Length: 237, dtype: float64

# Saving the cleaned data to CSV

Final dataframe:

In [22]:
df_combined = df_combined.reset_index(drop=True)
df_combined

,project,COM-1,COM-2,POP-1,STA-1,STA-2,STA-3,STA-4,STA-5,STA-6,STA-7,STA-8,STA-9,TEC-1,TEC-2,TEC-3,TEC-4,SWQ-1,SWQ-2.1,SWQ-2.2,SWQ-2.3,SWQ-2.4,SWQ-2.5,SWQ-2.6,SWQ-2.7,init,end,frequency (weeks),project_url,status
0,awf,3.078801e+05,775,217,1.193703,0,11,0,1,True,0,15,1,84,27485,20,68,0.227390,0.00,0,0,0,1,0.000000,574,2011-07-08,2012-09-16,12,https://github.com/valr/awf,retired
1,Agila-Theme,5.486906e+06,0,554,2.182067,0,50,0,1,True,0,0,0,0,793,20,10,0.000000,0.00,0,0,8,0,87.263707,22,2004-09-29,2006-12-05,12,https://github.com/arvi/Agila-Theme,retired
2,SerializerTests,9.172118e+04,0,170,0.772074,0,22,0,1,False,0,0,0,0,51166,22,81,0.000000,0.00,2,0,0,1,11.027687,196,2010-09-21,2011-06-30,12,https://github.com/Alois-xx/SerializerTests,retired
3,incubator-retired-amaterasu,8.753167e+06,123,139,2.028747,-73,27,-1,2,False,3,24,9,426,65317,98,715,0.002359,22.69,2,0,0,1,16.620933,154,2017-09-07,2019-09-18,12,https://github.com/apache/incubator-retired-am...,retired
4,incubator-ariatosca,2.021894e+06,1977,103,1.842574,15,47,-26,2,False,-5,46,14,13,46687,39,166,0.059026,57.47,25,0,0,0,4.990767,85,2016-08-27,2018-07-01,12,https://github.com/apache/incubator-ariatosca,retired
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,Flink,4.324401e+06,150989,60172,9.579740,273,13102,70,16,False,56,4721,1628,12851,7828,6925,40618,0.017610,113.40,0,21,65,214,14.329816,436,2014-12-17,2024-07-16,12,https://github.com/apache/Flink,graduated
233,Pulsar,5.351959e+06,28047,31473,5.828884,-433,3537,-358,12,False,18,1231,649,7279,64666,3295,17031,0.015900,98.41,0,5,42,109,11.537613,327,2018-09-19,2024-07-18,12,https://github.com/apache/Pulsar,graduated
234,NetBeans,2.133789e+06,7325,6027,5.248460,17,835,-41,2,False,-19,417,194,5061,5772,1584,10109,0.003356,32.87,0,50,523,1844,26.921600,2498,2019-04-17,2024-07-16,12,https://github.com/apache/NetBeans,graduated
235,incubator-NuttX,1.342960e+06,2634,6071,1.667351,573,1059,-696,6,False,-13,377,197,2067,67554,6816,46970,0.000000,0.02,0,50,452,1230,36.079518,1513,2022-11-17,2024-07-18,12,https://github.com/apache/incubator-NuttX,graduated


In [23]:
df_combined.to_csv("all_projects.csv", index=False)